# Add Dependences

In [31]:
include("../../src/forces_func.jl")
include("../../src/struct_data.jl")
include("../../src/test.jl")
include("../../src/forces.jl")
include("../../src/neighbor.jl")
include("../../src/run_event.jl")

@make_struct_func Cubic

force_func (generic function with 1 method)

# Add Data

In [32]:
parameters = Model(
    Cubic(
        μ₁      = 0.05,
        rₘᵢₙ    = 2,
        rₘₐₓ    = 3
    ),
    Contractile(
        fₚ       = 0.1
    ),
    Time(
        t_f      = 100000, 
        dt       = 0.1
    ),
    Neighbor(
        n_knn    = 100,
        nn       = 100
    ),
    Geometry(
        R_agg    = 9,
        position = [
            0.0    0.0    0.0;
            0.0    0.0    0.0
        ]
    ),
    Simulation(
        n_text   = 200,
        path     = "Fusion.xyz"
    )
)

## Var before func
println("\n Loading Force Function")
CUDA.@time force(r) =  force_func(parameters.Force::Cubic, r)


 Loading Force Function
  0.000026 seconds (17 CPU allocations: 1.237 KiB)


force (generic function with 1 method)

## Starting Data

In [33]:
# Loading Data
# CUDA.@time X = Float32.(readdlm("$(Int(parameters.Geometry.R_agg)).xyz")[3:end,2:end]) |> cu
CUDA.@time  X = Float32.(readdlm("../../data/init/Sphere/$(Int(parameters.Geometry.R_agg)).xyz")[3:end,2:end]) |> cu

  0.012318 seconds (8.06 k CPU allocations: 317.672 KiB) (1 GPU allocation: 6.305 KiB, 0.10% memmgmt time)


538×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5  -4.62  -7.35
  0.5  -4.62  -7.35
 -2.5  -2.89  -7.35
 -0.5  -2.89  -7.35
  1.5  -2.89  -7.35
  3.5  -2.89  -7.35
 -3.5  -1.15  -7.35
 -1.5  -1.15  -7.35
  0.5  -1.15  -7.35
  2.5  -1.15  -7.35
  4.5  -1.15  -7.35
 -4.5   0.58  -7.35
 -2.5   0.58  -7.35
  ⋮           
  3.5  -0.58   7.35
 -3.5   1.15   7.35
 -1.5   1.15   7.35
  0.5   1.15   7.35
  2.5   1.15   7.35
  4.5   1.15   7.35
 -2.5   2.89   7.35
 -0.5   2.89   7.35
  1.5   2.89   7.35
  3.5   2.89   7.35
 -1.5   4.62   7.35
  0.5   4.62   7.35

In [34]:
# Generating Struct
CUDA.@time Agg = Aggregate(Neighbor(50, 12), PositionCell(X));

  0.122834 seconds (345.18 k CPU allocations: 20.681 MiB) (12 GPU allocations: 4.813 MiB, 0.05% memmgmt time)


## run_event -> one_aggregate

In [35]:
# # Simulation One Aggregate
# CUDA.@time one_aggregate(true, parameters, Cubic(1.0, 2.0, 3.0), Agg)

## run_event -> fusion

In [36]:
# Simulating Two Aggregate
CUDA.@time fusion(true, parameters, Cubic(100, 2.0, 3.0), Agg)

Finding Equilibrium in one Aggregate


Progress: 100%|█████████████████████████| Time: 0:02:52m


Finding Equilibrium in two Aggregate


Progress: 100%|█████████████████████████| Time: 0:09:49


762.814998 seconds (1.42 G CPU allocations: 88.494 GiB, 2.82% gc time) (27.72 M GPU allocations: 11.676 TiB, 14.24% memmgmt time)
